<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Project 4 - West Nile Virus Prediction (Kaggle) <br> [Part 1 of 4]

_Prepared by: Kho Guan Guo, Soh Sze Ron, Timothy Chan, 3 Apr 2023_

## Contents:
- [Overview](#Overview)
- [Data Dictionary](#Data-Dictionary)
- [Data Cleaning](#Data-Cleaning)

---
## Overview
---

#### Background

In 2002, the first human cases of West Nile virus were reported in Chicago. By 2004 the City of Chicago and the Chicago Department of Public Health (CDPH) had established a comprehensive surveillance and control program that is still in effect today.

Every week from late spring through the fall, mosquitos in traps across the city are tested for the virus. The results of these tests influence when and where the city will spray airborne pesticides to control adult mosquito populations.

West Nile virus, is typically spread by mosquitoes. In about 80% of infections people have few or no symptoms. About 20% of people develop a fever, headache, vomiting, or a rash. In less than 1% of people, encephalitis or meningitis occurs, with associated neck stiffness, confusion, or seizures. Recovery may take weeks to months. The risk of death among those in whom the nervous system is affected is about 10 percent. People can reduce their risk of WNV by using insect repellent and wearing long-sleeved shirts and long pants to prevent mosquito bites.

The cost of treatment for severe symtoms developed from WNV can be staggeringly high. Preventative measures like spraying of pesticides, can also be costly and potentially harmful to the environment. It is thus essential to manage the spread of WNV efficiently and effectively.

#### Problem Statement

We are employees of the Disease And Treatment Agency, division of Societal Cures In Epidemiology and New Creative Engineering (DATA-SCIENCE). Due to the recent epidemic of West Nile Virus in the Windy City, we've had the Department of Public Health set up a surveillance and control system. 

Our role is to derive an effective plan to deploy pesticides throughout the city:
1. create a model to predict the presence of WNV throughout various locations in the city of Chicago
2. do a cost benefit analysis to weigh cost of spraying pesticides against potential benefits

The model and the analysis should serve to help the city better manage resources when dealing with WNV.

---
## Data Dictionary
---

#### **train.csv , test.csv**

  1. The train set consists of data from 2007, 2009, 2011, and 2013
  2. The test set consists of data from 2008, 2010, 2012 and 2014 (without data on NumMosquitos and WnvPresent)

| Feature                | Data Type | Description                                                                                              |
|------------------------|-----------|----------------------------------------------------------------------------------------------------------|
| Date                   | object    | Date that the WNV test is performed                                                                      |
| Species                | object    | The species of mosquitos                                                                       
| Trap                   | object    | Id of the trap                                                                            
| Latitude               | float     | Latitude returned from GeoCoder                                                                          |
| Longitude              | float     | Longitude returned from GeoCoder                                                                         |
| AddressAccuracy        | int       | Accuracy returned from GeoCoder                                                                          |
| NumMosquitos           | int       | Number of mosquitoes caught in this trap                                                                 |
| WnvPresent             | int       | Whether West Nile Virus was present in these mosquitos. 1 means WNV is present, and 0 means not present. |

#### **spray.csv** - GIS data of spraying efforts in 2011 and 2013

| Feature   | Data Type | Description            |
|-----------|-----------|------------------------|
| Date      | object    | The date of the spray  |
| Time      | object    | The time of the spray  |
| Latitude  | float     | Latitude of the spray  |
| Longitude | float     | Longitude of the spray |

In [1]:
# Importing Libraries

import numpy as np
import pandas as pd
import re

---
## Data Cleaning
---

### Import our training data

In [2]:
train = pd.read_csv('../data/train.csv', parse_dates=True)
test = pd.read_csv('../data/test.csv', parse_dates=True)
weather = pd.read_csv('../data/weather.csv', parse_dates=True)
spray = pd.read_csv('../data/spray.csv', parse_dates=True)

In [3]:
print(train.isnull().sum())
print()
print(test.isnull().sum())

Date                      0
Address                   0
Species                   0
Block                     0
Street                    0
Trap                      0
AddressNumberAndStreet    0
Latitude                  0
Longitude                 0
AddressAccuracy           0
NumMosquitos              0
WnvPresent                0
dtype: int64

Id                        0
Date                      0
Address                   0
Species                   0
Block                     0
Street                    0
Trap                      0
AddressNumberAndStreet    0
Latitude                  0
Longitude                 0
AddressAccuracy           0
dtype: int64


In [4]:
print(weather.isnull().sum())
print()
print(spray.isnull().sum())

Station        0
Date           0
Tmax           0
Tmin           0
Tavg           0
Depart         0
DewPoint       0
WetBulb        0
Heat           0
Cool           0
Sunrise        0
Sunset         0
CodeSum        0
Depth          0
Water1         0
SnowFall       0
PrecipTotal    0
StnPressure    0
SeaLevel       0
ResultSpeed    0
ResultDir      0
AvgSpeed       0
dtype: int64

Date           0
Time         584
Latitude       0
Longitude      0
dtype: int64


In [5]:
dataframes = [train, test, weather, spray]

# Converting column names to snake case
def camel_to_snake(name):
    name = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    return re.sub('([a-z0-9])([A-Z])', r'\1_\2', name).lower()

for df in dataframes:
    df.columns = [camel_to_snake(col) for col in df.columns]

### Train and Test Data

In [6]:
train.head()

,date,address,species,block,street,trap,address_number_and_street,latitude,longitude,address_accuracy,num_mosquitos,wnv_present
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,1,0
3,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,1,0
4,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,4,0


In [7]:
test.head()

,id,date,address,species,block,street,trap,address_number_and_street,latitude,longitude,address_accuracy
0,1,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
1,2,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
2,3,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
3,4,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX SALINARIUS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
4,5,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX TERRITANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9


In [8]:
columns_to_drop = ['address', 'block', 'street', 'address_accuracy']
train = train.drop(columns=columns_to_drop)
test = test.drop(columns=columns_to_drop)

In [9]:
# check for duplicates for train dataset
train.duplicated().value_counts()

False    9693
True      813
dtype: int64

Per Kaggle instructions:
*"These test results are organized in such a way that when the number of mosquitos exceed 50, they are split into another record (another row in the dataset), such that the number of mosquitos are capped at 50."*

We only dedupe for those with number of mosquitoes not equals to 50 (106 rows). The remaining were aggregated.

In [10]:
#Extracting the duplicated rows
train_dup = train[train.duplicated(keep='first')]

print(f"Number of mosquitoes in now not equals to 50: {train_dup[train_dup['num_mosquitos'] != 50].shape}")
print(f"Number of mosquitoes in row equals to 50: {train_dup[train_dup['num_mosquitos'] == 50].shape}")

Number of mosquitoes in now not equals to 50: (106, 8)
Number of mosquitoes in row equals to 50: (707, 8)


In [11]:
#Masking for only those that has 'NumMosquitos' less than 50
train_dup_less = train_dup[train_dup['num_mosquitos'] != 50]

#Dropping the duplicated rows by row index
train.drop(list(train_dup_less.index), axis = 0, inplace=True)

#Resetting index
train.reset_index(inplace=True, drop = True)

In [12]:
# check for duplicates again
train.duplicated().value_counts()

False    9693
True      707
dtype: int64

In [13]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10400 entries, 0 to 10399
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   date                       10400 non-null  object 
 1   species                    10400 non-null  object 
 2   trap                       10400 non-null  object 
 3   address_number_and_street  10400 non-null  object 
 4   latitude                   10400 non-null  float64
 5   longitude                  10400 non-null  float64
 6   num_mosquitos              10400 non-null  int64  
 7   wnv_present                10400 non-null  int64  
dtypes: float64(2), int64(2), object(4)
memory usage: 650.1+ KB


In [14]:
# aggregate the num_mosquitos and wnv_present
train = train.groupby(['date', 'species', 'trap', 'address_number_and_street', 'latitude', 'longitude']).sum().reset_index()

# reset wnv_present as binary
train['wnv_present'] = train['wnv_present'].apply(lambda x: 1 if x > 0 else 0)

In [15]:
# check for duplicates again
train.duplicated().value_counts()

False    8475
dtype: int64

In [16]:
# check for duplicates for test dataset
test.duplicated().value_counts()

False    116293
dtype: int64

In [17]:
# change date column dtype
train['date'] = pd.to_datetime(train['date'])
test['date'] = pd.to_datetime(test['date'])

### Weather Data

In [18]:
weather.head()

,station,date,tmax,tmin,tavg,depart,dew_point,wet_bulb,heat,cool,...,code_sum,depth,water1,snow_fall,precip_total,stn_pressure,sea_level,result_speed,result_dir,avg_speed
0,1,2007-05-01,83,50,67,14,51,56,0,2,...,,0,M,0.0,0.00,29.10,29.82,1.7,27,9.2
1,2,2007-05-01,84,52,68,M,51,57,0,3,...,,M,M,M,0.00,29.18,29.82,2.7,25,9.6
2,1,2007-05-02,59,42,51,-3,42,47,14,0,...,BR,0,M,0.0,0.00,29.38,30.09,13.0,4,13.4
3,2,2007-05-02,60,43,52,M,42,47,13,0,...,BR HZ,M,M,M,0.00,29.44,30.08,13.3,2,13.4
4,1,2007-05-03,66,46,56,2,40,48,9,0,...,,0,M,0.0,0.00,29.39,30.12,11.7,7,11.9


According to the weather data documentation, cells with 'M' indicate that the data is missing or not normally recorded by the station. We will replace these with null values to illustrate the sparsity of the data.

In [19]:
# Replace 'M' and '-' with NaN
weather = weather.replace('-', np.nan)
weather = weather.replace('M', np.nan)

In [20]:
weather.isnull().sum().sort_values(ascending=False)

water1          2944
sunset          1472
depth           1472
depart          1472
snow_fall       1472
sunrise         1472
tavg              11
heat              11
cool              11
sea_level          9
stn_pressure       4
wet_bulb           4
avg_speed          3
precip_total       2
code_sum           0
date               0
dew_point          0
tmin               0
tmax               0
result_speed       0
result_dir         0
station            0
dtype: int64

'Water1' will be dropped as 100% of its values are missing. 'SnowFall' and 'Depth' will be dropped as data collected in train and test sets are from May - October, and these features are only recorded during the winter months. 
Missing values from 'Tavg' will be calculated using average of 'Tmax' and 'Tmin'. All other missing values will be forward filled with most recently observed value in the same column. This way, we can maintain continuity in data and avoid distortion in our analysis. 

In [21]:
# Drop Water1, SnowFall, Depth, CodeSum
weather.drop(['water1', 'snow_fall', 'depth', 'code_sum', 'heat', 'cool'], axis=1, inplace=True)

In [22]:
# Fill missing values with forward fill
weather[['depart', 'sunrise', 'sunset', 'sea_level', 'wet_bulb', 'stn_pressure', 'avg_speed', 'precip_total']] \
= weather[['depart', 'sunrise', 'sunset', 'sea_level', 'wet_bulb', 'stn_pressure', 'avg_speed', 'precip_total']].fillna(method='ffill')

In [23]:
def replace_missing_Tavg(weather):
    # calculate the average of Tmax and Tmin
    avg_temp = round((weather['tmax'] + weather['tmin']) / 2,0)
    
    # replace missing values in Tavg with the calculated average
    weather['tavg'] = np.where(weather['tavg'].isnull(), avg_temp, weather['tavg'])
    
    # convert Tavg to integer data type
    weather['tavg'] = weather['tavg'].astype(int)
    
    return weather

In [24]:
weather = replace_missing_Tavg(weather)

In [25]:
# replace values which contain 'T' in 'PrecipTotal' with 0
weather['precip_total'] = weather['precip_total'].apply(lambda x: 0 if 'T' in str(x) else x)

In [26]:
# convert columns to integer dtype
weather[['tavg', 'wet_bulb']] = weather[['tavg', 'wet_bulb']].astype(int)

In [27]:
# convert columns to float dtype
weather[['precip_total', 'stn_pressure', 'sea_level', 'avg_speed']] = weather[['precip_total', 'stn_pressure', 'sea_level', 'avg_speed']].astype(float)

In [28]:
# convert date column to datetime dtype
weather['date'] = pd.to_datetime(weather['date'])

In [29]:
weather.duplicated().value_counts()

False    2944
dtype: int64

In [30]:
# find mean of avg temperature of each station
weather.groupby('station')['tavg'].mean()

station
1    66.606658
2    67.872283
Name: tavg, dtype: float64

In [31]:
# find mean of highest temperature of each station
weather.groupby('station')['tmax'].mean()

station
1    75.978940
2    76.353261
Name: tmax, dtype: float64

In [32]:
# find mean of highest temperature of each station
weather.groupby('station')['dew_point'].mean()

station
1    53.377717
2    53.538043
Name: dew_point, dtype: float64

In [33]:
# find mean of average wind speed of each station
weather.groupby('station')['avg_speed'].mean()

station
1    8.570788
2    8.587364
Name: avg_speed, dtype: float64

Since there is not much difference between Station 1 and Station 2, and Station 2 has missing data which needs to be imputed, we will just use Station 1 data which is more complete.

In [34]:
weather = weather[weather['station'] != 2]

In [35]:
def relative_humidity(tavg, wet_bulb):
    rh = 100 - (25/9)*(tavg - wet_bulb)
    return round(rh, 1)

In [36]:
weather['relative_humidity'] = weather.apply(lambda x: relative_humidity(x['tavg'], x['wet_bulb']), axis=1)

In [37]:
# Extract the hour and minute values from the 'sunrise' and 'sunset' columns
weather_temp = pd.DataFrame()
weather_temp['sunrise_hour'] = weather['sunrise'].astype(str).str[:-2].astype(int)
weather_temp['sunrise_minute'] = weather['sunrise'].astype(str).str[-2:].astype(int)
weather_temp['sunset_hour'] = weather['sunset'].astype(str).str[:-2].astype(int)
weather_temp['sunset_minute'] = weather['sunset'].astype(str).str[-2:].astype(int)

# Calculate the difference in minutes between sunrise and sunset
weather['minutes_between'] = (weather_temp['sunset_hour'] * 60 + weather_temp['sunset_minute']) - (weather_temp['sunrise_hour'] * 60 + weather_temp['sunrise_minute'])

In [38]:
weather.head()

,station,date,tmax,tmin,tavg,depart,dew_point,wet_bulb,sunrise,sunset,precip_total,stn_pressure,sea_level,result_speed,result_dir,avg_speed,relative_humidity,minutes_between
0,1,2007-05-01,83,50,67,14,51,56,0448,1849,0.0,29.10,29.82,1.7,27,9.2,69.4,841
2,1,2007-05-02,59,42,51,-3,42,47,0447,1850,0.0,29.38,30.09,13.0,4,13.4,88.9,843
4,1,2007-05-03,66,46,56,2,40,48,0446,1851,0.0,29.39,30.12,11.7,7,11.9,77.8,845
6,1,2007-05-04,66,49,58,4,41,50,0444,1852,0.0,29.31,30.05,10.4,8,10.8,77.8,848
8,1,2007-05-05,66,53,60,5,38,49,0443,1853,0.0,29.40,30.10,11.7,7,12.0,69.4,850


In [39]:
weather.isnull().sum().sort_values(ascending=False)

station              0
date                 0
relative_humidity    0
avg_speed            0
result_dir           0
result_speed         0
sea_level            0
stn_pressure         0
precip_total         0
sunset               0
sunrise              0
wet_bulb             0
dew_point            0
depart               0
tavg                 0
tmin                 0
tmax                 0
minutes_between      0
dtype: int64

In [40]:
weather.dtypes

station                       int64
date                 datetime64[ns]
tmax                          int64
tmin                          int64
tavg                          int64
depart                       object
dew_point                     int64
wet_bulb                      int64
sunrise                      object
sunset                       object
precip_total                float64
stn_pressure                float64
sea_level                   float64
result_speed                float64
result_dir                    int64
avg_speed                   float64
relative_humidity           float64
minutes_between               int64
dtype: object

### Spray Data

In [41]:
spray.head()

,date,time,latitude,longitude
0,2011-08-29,6:56:58 PM,42.391623,-88.089163
1,2011-08-29,6:57:08 PM,42.391348,-88.089163
2,2011-08-29,6:57:18 PM,42.391022,-88.089157
3,2011-08-29,6:57:28 PM,42.390637,-88.089158
4,2011-08-29,6:57:38 PM,42.390410,-88.088858


In [42]:
spray.isnull().sum()

date           0
time         584
latitude       0
longitude      0
dtype: int64

As the 584 missing values all fall between 7:44:32pm and 7:46:30pm on 7 Sep 2011, they will be filled with a time that falls in between, ie. 7:45:00pm

In [43]:
# fill NA values with '7:45:00 PM'
spray['time'].fillna('7:45:00 PM', inplace=True)

In [44]:
# Display counts for each date
spray['date'].value_counts()

2013-08-15    2668
2013-08-29    2302
2013-07-17    2202
2011-09-07    2114
2013-07-25    1607
2013-08-22    1587
2013-08-08    1195
2013-09-05     924
2013-08-16     141
2011-08-29      95
Name: date, dtype: int64

In [45]:
# check for duplicates
spray.duplicated().value_counts()

False    14294
True       541
dtype: int64

In [46]:
# drop duplicates
spray.drop_duplicates(inplace = True)

In [47]:
# check for duplicates again
spray.duplicated().value_counts()

False    14294
dtype: int64

As Spray only occurs in 2011 and 2013 from July to September, it may not be representative of the entire dataset and is not suitable for model training.

### Merge 'Train' and 'Weather' datasets

In [48]:
train_clean = pd.merge(train, weather, on='date')

In [49]:
import datetime

# Define a function to extract the week number from a date string
def get_week_number(date):
    date_str = date.strftime('%Y-%m-%d')
    date_obj = datetime.datetime.strptime(date_str, '%Y-%m-%d')
    return date_obj.isocalendar()[1]
    
# Apply the function to the date column of the DataFrame
train_clean['week_number'] = train_clean['date'].apply(get_week_number)

In [50]:
# split month and year from date
train_clean['month'] = train_clean['date'].dt.month
train_clean['year'] = train_clean['date'].dt.year

In [51]:
# concatenate columns 'latitude' and 'longitude' to create a new column 'coords'
train_clean['coords'] = train_clean['latitude'].astype(str) + ',' + train_clean['longitude'].astype(str)

In [52]:
train_clean.head()

,date,species,trap,address_number_and_street,latitude,longitude,num_mosquitos,wnv_present,station,tmax,...,sea_level,result_speed,result_dir,avg_speed,relative_humidity,minutes_between,week_number,month,year,coords
0,2007-05-29,CULEX PIPIENS,T096,"2200 W 89TH ST, Chicago, IL",41.731922,-87.677512,1,0,1,88,...,30.11,5.8,18,6.5,75.0,896,22,5,2007,"41.731922,-87.677512"
1,2007-05-29,CULEX PIPIENS/RESTUANS,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,1,0,1,88,...,30.11,5.8,18,6.5,75.0,896,22,5,2007,"41.95469,-87.800991"
2,2007-05-29,CULEX PIPIENS/RESTUANS,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,1,0,1,88,...,30.11,5.8,18,6.5,75.0,896,22,5,2007,"41.974089,-87.824812"
3,2007-05-29,CULEX PIPIENS/RESTUANS,T048,"1100 W ROOSEVELT, Chicago, IL",41.867108,-87.654224,1,0,1,88,...,30.11,5.8,18,6.5,75.0,896,22,5,2007,"41.867108,-87.654224"
4,2007-05-29,CULEX PIPIENS/RESTUANS,T050,"2100 N STAVE ST, Chicago, IL",41.919343,-87.694259,1,0,1,88,...,30.11,5.8,18,6.5,75.0,896,22,5,2007,"41.919343,-87.694259"


### Merge 'Test' and 'Weather' datasets

In [53]:
test_clean = pd.merge(test, weather, on='date')

In [54]:
# Apply the function to the date column of the DataFrame
test_clean['week_number'] = test_clean['date'].apply(get_week_number)

In [55]:
# split month and year from date
test_clean['month'] = test_clean['date'].dt.month
test_clean['year'] = test_clean['date'].dt.year

In [56]:
# concatenate columns 'latitude' and 'longitude' to create a new column 'coords'
test_clean['coords'] = test_clean['latitude'].astype(str) + ',' + test_clean['longitude'].astype(str)

In [57]:
test_clean.head()

,id,date,species,trap,address_number_and_street,latitude,longitude,station,tmax,tmin,...,sea_level,result_speed,result_dir,avg_speed,relative_humidity,minutes_between,week_number,month,year,coords
0,1,2008-06-11,CULEX PIPIENS/RESTUANS,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,1,86,61,...,29.99,8.9,18,10.0,72.2,910,24,6,2008,"41.95469,-87.800991"
1,2,2008-06-11,CULEX RESTUANS,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,1,86,61,...,29.99,8.9,18,10.0,72.2,910,24,6,2008,"41.95469,-87.800991"
2,3,2008-06-11,CULEX PIPIENS,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,1,86,61,...,29.99,8.9,18,10.0,72.2,910,24,6,2008,"41.95469,-87.800991"
3,4,2008-06-11,CULEX SALINARIUS,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,1,86,61,...,29.99,8.9,18,10.0,72.2,910,24,6,2008,"41.95469,-87.800991"
4,5,2008-06-11,CULEX TERRITANS,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,1,86,61,...,29.99,8.9,18,10.0,72.2,910,24,6,2008,"41.95469,-87.800991"


### Save cleaned data

In [59]:
train_clean.to_csv('../data/data_cleaned/train_clean.csv', index=False)

In [60]:
test_clean.to_csv('../data/data_cleaned/test_clean.csv', index=False)

In [61]:
spray.to_csv('../data/data_cleaned/spray_clean.csv', index=False)

In [62]:
train_clean.head()

,date,species,trap,address_number_and_street,latitude,longitude,num_mosquitos,wnv_present,station,tmax,...,sea_level,result_speed,result_dir,avg_speed,relative_humidity,minutes_between,week_number,month,year,coords
0,2007-05-29,CULEX PIPIENS,T096,"2200 W 89TH ST, Chicago, IL",41.731922,-87.677512,1,0,1,88,...,30.11,5.8,18,6.5,75.0,896,22,5,2007,"41.731922,-87.677512"
1,2007-05-29,CULEX PIPIENS/RESTUANS,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,1,0,1,88,...,30.11,5.8,18,6.5,75.0,896,22,5,2007,"41.95469,-87.800991"
2,2007-05-29,CULEX PIPIENS/RESTUANS,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,1,0,1,88,...,30.11,5.8,18,6.5,75.0,896,22,5,2007,"41.974089,-87.824812"
3,2007-05-29,CULEX PIPIENS/RESTUANS,T048,"1100 W ROOSEVELT, Chicago, IL",41.867108,-87.654224,1,0,1,88,...,30.11,5.8,18,6.5,75.0,896,22,5,2007,"41.867108,-87.654224"
4,2007-05-29,CULEX PIPIENS/RESTUANS,T050,"2100 N STAVE ST, Chicago, IL",41.919343,-87.694259,1,0,1,88,...,30.11,5.8,18,6.5,75.0,896,22,5,2007,"41.919343,-87.694259"
